In [321]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

pattern = cv2.imread('../imgs/pattern.png')
targets_loc = "IMG_6719.JPG IMG_6720.JPG IMG_6721.JPG IMG_6722.JPG IMG_6723.JPG IMG_6724.JPG IMG_6725.JPG IMG_6726.JPG IMG_6727.JPG"
targets_loc = targets_loc.split(' ')
# print(targets_loc)
for s in range(len(targets_loc)):
    targets_loc[s] = '../imgs/' + targets_loc[s]
# print(targets_loc)
targets = [cv2.imread(s) for s in targets_loc]

NATIVE_RES = targets[0].shape

#note units in cm
TARGET_DIM = (88, 88)
FOCAL_LEN = 4.2 #in mm

def get_tri_pts(image):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    ret,edge = cv2.threshold(gray,190,255,cv2.THRESH_BINARY)
#     edge = cv2.Canny(gray,100,200)
    #now find contours and than we can sort for the target
    im2, contours, hierarchy = cv2.findContours(edge,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(edge, contours
                     , -1, (255,255,255), 3)
    mholes = sorted(contours, key=cv2.contourArea)[-6:-2]
    tri = []
    for c in mholes:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        if (cX,cY) not in tri:
            tri.append((cX,cY,0,1))
            
    if len(tri) > 3:
        temp = []
        for i in range(len(tri)):
            x = tri[i]
            for j in range(i,len(tri)):
                y = tri[j]
                
#                 print("{} {} {}".format(x,y,x[0] - y[0] > 5 or x[1] - y[1] > 5))
                if abs(x[0] - y[0]) >= 5 or abs(x[1] - y[1]) >= 5  :
#                     print("{} {} {}".format(x,y,x[0] - y[0] > 5 or x[1] - y[1] > 5))
                    if x not in temp:
                        temp.append(x)
                    if y not in temp:
                        temp.append(y)
                elif len(temp) == 3:
                    break
            if len(temp) == 3:
                break
        return temp
    
PATTERN_PTS = get_tri_pts(pattern)
pc = 'A'
for p in PATTERN_PTS:
    cv2.circle(pattern, p[:2], 7, (255,255,0), -1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(pattern,pc,p[:2], font, 3,(55,55,255),2,cv2.LINE_AA)
    pc = chr(ord(pc) + 1)
# print(PATTERN_PTS)


def find_pattern(image,marker):
    
    #should change approach since this is a giant white piece of paper =P
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    ret,edge = cv2.threshold(gray,190,255,cv2.THRESH_BINARY)
#     edge = cv2.Canny(gray,100,200)
    #now find contours and than we can sort for the target
    im2, contours, hierarchy = cv2.findContours(edge,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(edge, contours
                     , -1, (255,255,255), 3)
    
    #find the square and from there we can create a mask to extract an roi
#     c = max(contours, key = cv2.contourArea)

    curr_max = [0,0] #(size,index)
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if area > curr_max[0]:
            curr_max = [area,i]
            
#     c = contours[curr_max[1]]
    hierarchy = hierarchy[curr_max[1]]
# #     print(hiearchy[curr_max[1],2])
    holes = [contours[i] for i in range(len(contours)) if hierarchy[i][3] >= 0]
    mholes = sorted(holes, key=cv2.contourArea)[-5:-1]
    holes_i = [i for i in range(len(contours)) if hierarchy[i][3] >= 0]

    holes = np.vstack(holes).squeeze()

#     x,y,
    rect = cv2.minAreaRect(holes)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    theta = rect[-1]
#     cv2.drawContours(image,holes,-,(0,0,255),3)
    
    
    
    
    x,y,width,height = cv2.boundingRect(box)
    target = image[y:y+int(height), x:x+int(width) ]
    width = int(rect[1][0])
    height = int(rect[1][1])
    tri = get_tri_pts(target)
    

    height,width = target.shape[:2]
    
    #determine if two points are below or above mid point
    def det_above(height,tri):
        cnt = 0
#         print(height//2)
        for t in tri:
            if t[1] < height//2:
                cnt += 1
        return cnt == 2
    
    def det_left(width,tri):
        cnt = 0
#         print(width//2)
        for t in tri:
            if t[0] < width//2:
                cnt += 1
        return cnt == 2
        
    two_above = det_above(height,tri)
    one_left = det_left(width,tri)
    #if two above and one left than just figure out the angle 
#     print(two_above,one_left)
    if two_above and one_left:
        if theta < -45:
            theta = -(90 + theta)
 
        else:
            theta = -theta
    elif two_above and not one_left:
        #Min rotation of at least 90
        theta = -(90 + theta)
    elif not two_above and one_left:
        #TODO
#         print(theta)
        if theta < -45:
            theta = (-theta)
 
        else:
            theta = -theta + 90
    elif not two_above and not one_left:
        #TODO
#         print(theta)
        if theta < -45:
            theta = -theta
 
        else:
            theta = -theta + 180
#         theta = -(180 +theta)
    #theta is the orientation of the camera

    
    
    return (image,  int(rect[1][0]),  int(rect[1][1]), theta, box)
#     return image

def calc_abs_distancek(height,f,pheight, target_shape):
    #will calculate distance from pattern to camera
#     4.80 x 3.60 mm width height
    #Will be inaccurate due to angle and use of triangle similarity
    
    return (TARGET_DIM[0] * FOCAL_LEN)/3.6

#calibration matrix estimated
#going to approximate using values from apples dev documentation
# fx = (targets[0].shape[1]/2) / math.tan(58.040/2)
# fy = (targets[0].shape[0]/2) / math.tan(58.040/2)
# fx = 4.2
# fy = 4.2
# camera_mat = np.array([
#     [fx,0,targets[0].shape[1]/2],
#     [0,fx,targets[0].shape[0]/2],
#     [0,0,1]
# ])
# print(camera_mat)


# RMS: 1.13707201375
camera_mat = np.array(
[[  2.80360356e+03  , 0.00000000e+00  , targets[0].shape[1]/2],
 [  0.00000000e+00  , 2.80521893e+03  , targets[0].shape[1]/2],
 [  0.00000000e+00  , 0.00000000e+00  , 1.00000000e+00]])


cropped = []
cv2.imwrite('pattern_pts.jpg',pattern)
obj_pts = np.zeros((4,3,1))
# obj_pts = [
#     [-.044,-.044,0],
#     [-.044, .044,0],
#     [ .044,-.044,0],
#     [0.044, 0.044,0]
# ]

obj_pts[0] = [[-4.4], [-4.4],[0]]
obj_pts[1] = [[-4.4], [4.4],[0]]
obj_pts[2] = [[4.4], [-4.4],[0]]
obj_pts[3] = [[4.4], [4.4],[0]]

#estimate f_x and f_y avg
m = 2803/FOCAL_LEN
# print(m)
DIAG_LEN = (targets[0]**2 + (targets[1]**2))**.5

# obj_pts = np.ascontiguousarray(obj_pts,dtype=np.float64)
# obj_pts = np.reshape(obj_pts,(4,1,3))
for img in range(len(targets)):
    image,width,height,theta,box = find_pattern(targets[img],pattern)
    #since target is 8.8cm 8.8cm
    #top left starting from 0
    
    image_pts = np.zeros((4,2,1))
#     image_pts[0] = [[tri[0][0]],[tri[0][1]]]
#     image_pts[1] = [[tri[1][0]],[tri[1][1]]]
#     image_pts[2] = [[tri[2][0]],[tri[2][1]]]
    
    image_pts[0] = [[box[0][0]], [box[0][1]]]
    image_pts[1] = [[box[1][0]], [box[1][1]]]
    image_pts[2] = [[box[2][0]] ,[box[2][1]]]
    image_pts[3] = [[box[3][0]] ,[box[3][1]]]


    distCoeffs = np.zeros((5,1))
    
    
    found, rvec, tvec = cv2.solvePnP(obj_pts, image_pts, camera_mat, distCoeffs)
    rot_m = cv2.Rodrigues(rvec)[0]
#     print(rot_m)
    
#     print(height)
    distance_mm = (TARGET_DIM[0] * FOCAL_LEN)/(width / m)
    #Very Likely to be inaccurate due to assumptions taken
    print("For picture {}".format(targets_loc[img]))
    print("Distance from camera {} mm".format(distance_mm))
    print("XY Angle {} degrees".format(theta))
    print("")
    
    
    #use that and theta to determine x y
    
    #estimate gamma angle between target and camera
    #assume camera has 55 angle
    #get diagonal length
    pix_d = DIAG_LEN / 55
    
    cx = (box[0][0] + box[1][0] + box[2][0] + box[3][0])/4
    cy = (box[0][1] + box[1][1] + box[2][1] + box[3][1])/4
    

#     cv2.imwrite('roi{}-{}x{}.jpg'.format(img,width,height),image)
# plt.imshow(targets[0])
print('done')







For picture ../imgs/IMG_6719.JPG
Distance from camera 927.3082706766919 mm
XY Angle -32.48703384399414 degrees

For picture ../imgs/IMG_6720.JPG
Distance from camera 868.5352112676056 mm
XY Angle -89.34893959760666 degrees

For picture ../imgs/IMG_6721.JPG
Distance from camera 859.4564459930315 mm
XY Angle 214.93543243408203 degrees

For picture ../imgs/IMG_6722.JPG
Distance from camera 1067.8095238095239 mm
XY Angle 84.63226318359375 degrees

For picture ../imgs/IMG_6723.JPG
Distance from camera 906.8529411764706 mm
XY Angle 125.7538833618164 degrees

For picture ../imgs/IMG_6724.JPG
Distance from camera 1067.8095238095239 mm
XY Angle 88.85993194580078 degrees

For picture ../imgs/IMG_6725.JPG
Distance from camera 574.9743589743591 mm
XY Angle 9.462321281433105 degrees

For picture ../imgs/IMG_6726.JPG
Distance from camera 763.6656346749227 mm
XY Angle -10.304847717285156 degrees

For picture ../imgs/IMG_6727.JPG
Distance from camera 1019.2727272727274 mm
XY Angle -33.6900634765625 de